In [1]:
#importing the necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [2]:
#loading the dataset
df = pd.read_csv('rfm_data.csv')
df

,CustomerID,PurchaseDate,TransactionAmount,ProductInformation,OrderID,Location
0,8814,2023-04-11,943.31,Product C,890075,Tokyo
1,2188,2023-04-11,463.70,Product A,176819,London
2,4608,2023-04-11,80.28,Product A,340062,New York
3,2559,2023-04-11,221.29,Product A,239145,London
4,9482,2023-04-11,739.56,Product A,194545,Paris
...,...,...,...,...,...,...
995,2970,2023-06-10,759.62,Product B,275284,London
996,6669,2023-06-10,941.50,Product C,987025,New York
997,8836,2023-06-10,545.36,Product C,512842,London
998,1440,2023-06-10,729.94,Product B,559753,Paris


In [3]:
#getting information about the dataframe
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   CustomerID          1000 non-null   int64  
 1   PurchaseDate        1000 non-null   object 
 2   TransactionAmount   1000 non-null   float64
 3   ProductInformation  1000 non-null   object 
 4   OrderID             1000 non-null   int64  
 5   Location            1000 non-null   object 
dtypes: float64(1), int64(2), object(3)
memory usage: 215.0 KB


In [4]:
#describing the dataframe
df.describe().T

,count,mean,std,min,25%,50%,75%,max
CustomerID,1000.0,5554.78900,2605.014863,1011.00,3273.00,5538.000,7821.75,9991.00
TransactionAmount,1000.0,513.67781,286.098700,12.13,257.12,523.565,759.86,999.44
OrderID,1000.0,554071.39800,264695.448814,100096.00,313152.00,564671.500,783052.25,999695.00


In [5]:
#analyzing the dataframe to see the different locations in the dataset
df['Location'].unique()

array(['Tokyo', 'London', 'New York', 'Paris'], dtype=object)

In [6]:
#analyzing to see the different products
df['ProductInformation'].unique()

array(['Product C', 'Product A', 'Product B', 'Product D'], dtype=object)

In [7]:
#analyzing to see how many customers
df['CustomerID'].nunique()

946

In [8]:
#converting Purchase Date to datetime
df['PurchaseDate'] = pd.to_datetime(df['PurchaseDate'])

In [9]:
#calculating  customer recency
recency = df.groupby('CustomerID')['PurchaseDate'].max().reset_index()
recency['recency'] = recency['PurchaseDate'].max() - recency['PurchaseDate']
recency.head()

,CustomerID,PurchaseDate,recency
0,1011,2023-05-08,33 days
1,1025,2023-05-20,21 days
2,1029,2023-06-10,0 days
3,1046,2023-04-28,43 days
4,1049,2023-05-28,13 days


In [14]:
#calculating customer frequency
frequency = df.drop_duplicates().groupby('CustomerID').size().reset_index(name='Frequency')
frequency.columns = ['CustomerID', 'Frequency']
frequency

,CustomerID,Frequency
0,1011,2
1,1025,1
2,1029,1
3,1046,1
4,1049,1
...,...,...
941,9941,1
942,9950,1
943,9954,1
944,9985,1


In [15]:
#calculating monetary value
monetary = df.groupby('CustomerID')['TransactionAmount'].sum().reset_index()
monetary.columns = ['CustomerID', 'monetary']
monetary.head()

,CustomerID,monetary
0,1011,1129.02
1,1025,359.29
2,1029,704.99
3,1046,859.82
4,1049,225.72


In [17]:
#merging recency, frequency and monetary into one column
rfm = recency.merge(frequency, on='CustomerID', how='left')
rfm_df = rfm.merge(monetary, on='CustomerID', how='left').drop(columns='PurchaseDate')
rfm_df

,CustomerID,recency,Frequency,monetary
0,1011,33 days,2,1129.02
1,1025,21 days,1,359.29
2,1029,0 days,1,704.99
3,1046,43 days,1,859.82
4,1049,13 days,1,225.72
...,...,...,...,...
941,9941,42 days,1,960.53
942,9950,38 days,1,679.11
943,9954,12 days,1,798.01
944,9985,57 days,1,36.10


In [21]:
#sorting the dataframe by calculating ranks and normalizing them
rfm_df['R_rank'] = rfm_df['recency'].rank(ascending=False)
rfm_df['F_rank'] = rfm_df['Frequency'].rank(ascending=True)
rfm_df['M_rank'] = rfm_df['monetary'].rank(ascending=True)
 
# normalizing the rank of the customers
rfm_df['R_rank_norm'] = (rfm_df['R_rank']/rfm_df['R_rank'].max())*100
rfm_df['F_rank_norm'] = (rfm_df['F_rank']/rfm_df['F_rank'].max())*100
rfm_df['M_rank_norm'] = (rfm_df['F_rank']/rfm_df['M_rank'].max())*100
 
rfm_df.drop(columns=['R_rank', 'F_rank', 'M_rank'], inplace=True)
 
rfm_df.head()

,CustomerID,recency,Frequency,monetary,R_rank_norm,F_rank_norm,M_rank_norm
0,1011,33 days,2,1129.02,47.331910,97.301587,97.198732
1,1025,21 days,1,359.29,64.727855,47.407407,47.357294
2,1029,0 days,1,704.99,100.000000,47.407407,47.357294
3,1046,43 days,1,859.82,27.908218,47.407407,47.357294
4,1049,13 days,1,225.72,78.762006,47.407407,47.357294
